In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from torch import nn,optim
from ray.air import session
import pandas as pd
from ray.tune.search.hyperopt import HyperOptSearch
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_percentage_error as MAPE
import ray
from ray import  tune
from ray.air import session
from sklearn.preprocessing import MinMaxScaler
from hyperopt import hp
import torch.nn as nn
from data_proc import normalized_dataset_test_0,normalized_dataset_train_0



In [21]:
class AdaptiveTanh(torch.nn.Module):
    def __init__(self, init_alpha=0.5, init_beta=0.5):
        self.alpha = torch.nn.Parameter(torch.tensor(init_alpha))
        self.beta = torch.nn.Parameter(torch.tensor(init_beta))
    def forward(self, x):
        return torch.tanh(self.alpha*x + self.beta) 



class Net(nn.Module):
    def __init__(self,L1,L2,L3,a,b,g,d):
        super(Net,self).__init__()
  
        self.layers = nn.Sequential(
      nn.Linear(6,L1),
      AdaptiveTanh(init_alpha=a, init_beta=b, init_gamma=g, init_delta=d),
      nn.Linear(L1,L2),
      AdaptiveTanh(init_alpha=a ,init_beta=b, init_gamma=g, init_delta=d),
      nn.Linear(L2,L3),
      AdaptiveTanh(init_alpha=a, init_beta=b, init_gamma=g, init_delta=d),
      nn.Linear(L3,1),
      AdaptiveTanh(init_alpha=a, init_beta=b, init_gamma=g, init_delta=d),
    )
    def forward(self, x):
        return self.layers(x)


        
      
    

model = Net(space["L1"],space["L2"],space["L3"],space["a"],space["b"],space["g"],space["d"])
#####MODEL#####


normalized_train = DataLoader(normalized_dataset_train_0, batch_size=space["batch_size"], shuffle=True)
normalized_test = DataLoader(normalized_dataset_test_0, batch_size=space["batch_size"], shuffle=False)

###PARAMETERS#####
criterion = nn.MSELoss()
optimizer = getattr(optim, space["optimizer"])(model.parameters(), lr=0.001, eps=space["eps"])
num_epochs = 100
###PARAMETERS#####



#####TRAINING LOOP#######
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(normalized_train):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
#####TRAINING LOOP#######    
###EVALUATION####       
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(normalized_test):
            output = model(data)
            test_loss += criterion(output, target).item()
        test_loss /= len(normalized_test)

0.19480536337941884
0.07718850998624936
0.0658252226397284
0.17459486588946918
0.16917814746387858
0.03615961024579384
0.07611925984218494
0.11299338337840206
0.0924734383599082
0.22519922340983775
0.08531427145866094
0.2803699368193968
0.14834721132612494
0.06530961287694413
0.10841692137434635
0.22689395521336528
0.19239222733720915
0.10872279042239065
0.18333237376898082
0.13643313391940362
0.05213250436528183
0.1064069890347657
0.16336738059102598
0.25544883754493586
0.15576234493793836
0.15649893030857862
0.125283076842107
0.12167660648753839
0.08011910661693607
0.3108891528266579
0.16941497506601597


KeyboardInterrupt: 